In [59]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

In [60]:
def get_game(url):
    bets = []
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    game_title = soup.find('span', class_='o_match-title').text.strip()
    time.sleep(8)  
   
    accordion_button = driver.find_element(By.XPATH, "//button[@class='o_accordion-item__trigger']//span[contains(text(), 'Genaues Ergebnis')]")
    accordion_button.click()

    time.sleep(2)  

    # Get the page source and parse it with BeautifulSoup


    # Find the parent accordion item of the clicked button
    accordion_item = accordion_button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'o_accordion-item')]")

    # Ensure the accordion item is expanded
    if 'o_accordion-item--expanded' in accordion_item.get_attribute('class'):
        # Find the corresponding panel within the expanded accordion item
        panel = accordion_item.find_element(By.CLASS_NAME, 'o_accordion-item__panel')

        # Get the HTML content of the panel and parse it with BeautifulSoup
        panel_html_content = panel.get_attribute('innerHTML')
        panel_soup = BeautifulSoup(panel_html_content, 'html.parser')

        # Locate the specific bet item section with the header "Genaues Resultat (reg.Spielzeit)"
        bet_items_section = panel_soup.find_all('div', class_='o_bet-item')

        for section in bet_items_section:
            header = section.find('div', class_='o_bet-item__header').text.strip()
            if "Genaues Resultat (reg.Spielzeit)" in header:
                # Extract all bet items within this section
                bet_items = section.find('div', class_='o_bet-item__content ng-star-inserted').find_all('button', class_='m_bet-button')
                for item in bet_items:
                    name = item.find('span', class_='m_bet-button__name').text.strip()
                    odds = item.find('span', class_='m_bet-button__odds-text').text.strip()
                    bets.append((name, odds))
                break  # Exit after finding the correct section to avoid processing further
    else:
        print("Accordion item is not expanded.")
    

    # Get the page source and parse it with BeautifulSoup
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    output = soup.find_all('span', class_='o_bet-item__content ng-star-inserted')
    for content in output:
        print(content.text)

    # Close the WebDriver
    driver.quit()
    return bets,game_title
    

In [61]:
all_games = []
with open('list_urls.txt', 'r') as file:
    all_games = [line.strip() for line in file.readlines()]

In [71]:
all_bets_data = []

for game_url in all_games:
    bets = []
    game_title = ""
    for i in range(5):
        try: 
            bets, game_title = get_game(game_url)
            if bets:
                break
        except Exception as e:
            print(f"Attempt {i+1} failed for game {game_url}: {e}")
    if bets:
        for name, odds in bets:
            all_bets_data.append([game_title, name, float(odds)])



In [75]:
df = pd.DataFrame(all_bets_data, columns=["Game Title", "Result", "Odds"])
# Sort the DataFrame by game title and odds
df_sorted = df.sort_values(by=["Game Title", "Odds"])

# Keep only the 3 lowest odds for each game title
df_filtered = df_sorted.groupby("Game Title").head(3)

# Display the filtered DataFrame
display(df_filtered)



,Game Title,Result,Odds
1,Deutschland – Schottland,2:0,5.11
0,Deutschland – Schottland,1:0,5.84
3,Deutschland – Schottland,3:0,6.54
